<a href="https://colab.research.google.com/github/VishnuPy/MyPython/blob/master/Spam_Detection_ML_Voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
%ls

Found GPU at: /device:GPU:0
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
'My Drive'/


In [4]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.446855557000049
GPU (s):
0.17170512300003793
GPU speedup over CPU: 20x


In [0]:
import zipfile
import random
import glob
import os
import sklearn
import sklearn.svm
import sklearn.naive_bayes
import sklearn.neighbors
import sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [0]:
def preprocess(data):
    """
    Function which removes
    not important characters
    """

    final_message =\
            data.strip("Subject").strip(":").strip("\n").strip("\r").strip("re").strip("-")

    return final_message

def get_features(messages):

    """
    Function which converts the
    messages into feature or vectorize form
    """
    vectorizer = CountVectorizer(min_df=1)
    X_count_vec = vectorizer.fit_transform(messages)
    transformer = TfidfTransformer(smooth_idf=False)
    X = transformer.fit_transform(X_count_vec)

    return X

def read_data(curr_dir):
    """
    Function which reads the
    two directories and preprocess
    and creates messages and labels list
    """

    messages_list = []
    temp_list = []
    messages = []
    labels = []

    # Reading Ham data
    ham_folder = glob.glob(curr_dir + "/dataset/ham" + "/*.txt")
    for file_item in ham_folder:
        with open(file_item, "r") as fp:
            ham_message = preprocess(fp.readline())
            messages_list.append({ham_message: 1})

    # Reading Spam data
    spam_folder = glob.glob(curr_dir + "/dataset/spam" + "/*.txt")
    for file_item in spam_folder:
        with open(file_item, "r") as fp:
            spam_message = preprocess(fp.readline())
            temp_list.append({spam_message: 0})
            messages_list.append({spam_message: 0})

    random_spam_mssgs = random.sample(temp_list, 1000)
    messages_list.extend(random_spam_mssgs)

    random.shuffle(messages_list)
    for item in messages_list:
        for key, value in item.items():
            messages.append(key)
            labels.append(value)

    return messages, labels


In [7]:
cd  "/gdrive/My Drive/MyLearning/MLDLAIPython/Data/TextData/"

/gdrive/My Drive/MyLearning/MLDLAIPython/Data/TextData


In [8]:
def test_classifier_nb(X_train, X_test, y_train, y_test, nb_clf):
    """
    Function which tests
    Naive Bayes classifier
    outputs the accuracy and confusion matrix
    """

    print("Naive Bayes\n")
    nb_clf.fit(X_train, y_train)
    y_predicted = nb_clf.predict(X_test)

    accuracy = accuracy_score(y_predicted, y_test)
    print("Naive Bayes Accuracy:", accuracy)
    print("Naive Bayes Classification Report\n")
    print(sklearn.metrics.classification_report(y_test, y_predicted,
                                                target_names=None))

    print("Naive Bayes Confusion Matrix\n")
    print(sklearn.metrics.confusion_matrix(y_test, y_predicted))


def test_classifier_svm(X_train, X_test, y_train, y_test, svm_clf):
    """
    Function which tests
    SVM classifier
    outputs the accuracy and confusion matrix
    """

    print("\n\nSupport Vector Machine\n")
    svm_clf.fit(X_train, y_train)
    y_predicted = svm_clf.predict(X_test)

    accuracy = accuracy_score(y_predicted, y_test)
    print("SVM Accuracy:", accuracy)
    print("SVM Classification Report\n")
    print(sklearn.metrics.classification_report(y_test, y_predicted,
                                                target_names=None))

    print("SVM Confusion Matrix\n")
    print(sklearn.metrics.confusion_matrix(y_test, y_predicted))


def test_classifier_knn(X_train, X_test, y_train, y_test, knn_clf):
    """
    Function which tests
    KNN classifier
    outputs the accuracy and confusion matrix
    """

    print("\n\nKNN\n")
    knn_clf.fit(X_train, y_train)
    y_predicted = knn_clf.predict(X_test)

    accuracy = accuracy_score(y_predicted, y_test)
    print("KNN Accuracy:", accuracy)
    print("KNN Classification Report\n")
    print(sklearn.metrics.classification_report(y_test, y_predicted,
                                                target_names=None))

    print("KNN Confusion Matrix\n")
    print(sklearn.metrics.confusion_matrix(y_test, y_predicted))


def test_classifier_dtree(X_train, X_test, y_train, y_test, dtree_clf):
    """
    Function which tests
    Decision Tree classifier
    outputs the accuracy and confusion matrix
    """

    print("\n\nDecision Tree\n")
    dtree_clf.fit(X_train, y_train)
    y_predicted = dtree_clf.predict(X_test)

    accuracy = accuracy_score(y_predicted, y_test)
    print("Decision Tree Accuracy:", accuracy)
    print("Decision Classification Report\n")
    print(sklearn.metrics.classification_report(y_test, y_predicted,
                                                target_names=None))

    print("Decision Tree Confusion Matrix\n")
    print(sklearn.metrics.confusion_matrix(y_test, y_predicted))


def test_classifier_rforest(X_train, X_test, y_train, y_test, rforest_clf):
    """
    Function which tests
    Random Forest classifier
    outputs the accuracy and confusion matrix
    """

    print("\n\nRandom Forest\n")
    rforest_clf.fit(X_train, y_train)
    y_predicted = rforest_clf.predict(X_test)

    accuracy = accuracy_score(y_predicted, y_test)
    print("Random Forest Accuracy:", accuracy)
    print("Random Forest Report\n")
    print(sklearn.metrics.classification_report(y_test, y_predicted,
                                                target_names=None))

    print("Random Forest Confusion Matrix\n")
    print(sklearn.metrics.confusion_matrix(y_test, y_predicted))


def test_classifier_voting(X_train, X_test, y_train, y_test, vote_clf):
    """
    Function which
    does hard voting of all classifiers
    outputs the accuracy and confusion matrix
    """

    print("\n\nVoting Classifier\n")
    vote_clf.fit(X_train, y_train)
    y_predicted = vote_clf.predict(X_test)

    accuracy = accuracy_score(y_predicted, y_test)
    print("Voting Classifier Accuracy:", accuracy)
    print("Voting Classifier Report\n")
    print(sklearn.metrics.classification_report(y_test, y_predicted,
                                                target_names=None))

    print("Voting Classifier Confusion Matrix\n")
    print(sklearn.metrics.confusion_matrix(y_test, y_predicted))


if __name__ == "__main__":

    n_neighbors = 11
    weights = 'distance'
    curr_dir = dir_path = os.path.dirname(os.path.realpath("__file__"))
    print ('curr directory -- > ', curr_dir)
    if os.path.isdir("dataset"):
        print("Directory Exists, no need of unzipping dataset")
    else:
        print("Zipping the datasets into two folders Spam and Ham")
        print ('curr_dirVVVVVV -- >', curr_dir)
        print (os.path.dirname(os.path.realpath("__file__")))
        zip_pointer = zipfile.ZipFile("dataset.zip", 'r')
        zip_pointer.extractall(curr_dir)
        zip_pointer.close()

    messages, labels = read_data(curr_dir)
    features = get_features(messages)
    X_train, X_test, y_train, y_test = train_test_split(features, labels,
                                                        test_size=0.33,
                                                        random_state=42)

    nb_clf = sklearn.naive_bayes.MultinomialNB()
    svm_clf = sklearn.svm.LinearSVC()
    knn_clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors,
                                                     weights=weights)
    dtree_clf = DecisionTreeClassifier(random_state=0)
    rforest_clf = RandomForestClassifier(n_estimators=10)
    vote_clf = VotingClassifier(estimators=[('nb', nb_clf), ('svm', svm_clf),
                                            #('knn', knn_clf),
                                            ('dt', dtree_clf),
                                            ('rt', rforest_clf)],
                                voting='hard')

    # testing the classifier

    test_classifier_nb(X_train, X_test, y_train, y_test,  nb_clf)
    test_classifier_svm(X_train, X_test, y_train, y_test,  svm_clf)
    #test_classifier_knn(X_train, X_test, y_train, y_test,  knn_clf)
    test_classifier_dtree(X_train, X_test, y_train, y_test,  dtree_clf)
    test_classifier_rforest(X_train, X_test, y_train, y_test, rforest_clf)
    test_classifier_voting(X_train, X_test, y_train, y_test, vote_clf)

curr directory -- >  /gdrive/My Drive/MyLearning/MLDLAIPython/Data/TextData
Directory Exists, no need of unzipping dataset
Naive Bayes

Naive Bayes Accuracy: 0.9158878504672897
Naive Bayes Classification Report

              precision    recall  f1-score   support

           0       0.95      0.85      0.89       847
           1       0.90      0.97      0.93      1186

    accuracy                           0.92      2033
   macro avg       0.92      0.91      0.91      2033
weighted avg       0.92      0.92      0.92      2033

Naive Bayes Confusion Matrix

[[ 717  130]
 [  41 1145]]


Support Vector Machine

SVM Accuracy: 0.9449090014756517
SVM Classification Report

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       847
           1       0.97      0.93      0.95      1186

    accuracy                           0.94      2033
   macro avg       0.94      0.95      0.94      2033
weighted avg       0.95      0.94      0.95   